## Slope Data Acquisition & Preprocess

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
crs = {'init': 'epsg:4326'}

#### Open the full climatology dataset

In [ ]:
df = pd.read_csv('Full_Climatology.csv')
del df['Unnamed: 0']
gdf = gpd.GeoDataFrame(df, crs=crs, geometry = gpd.points_from_xy(df.Longitude, df.Latitude))
del df

In [ ]:
#it is a huge data set with more than 1.5 million datapoints
#filter the gdf and keep only the entries with frd > 5
gdf = gdf[gdf.VHRFC_LIS_FRD > 5]
gdf

#### Connect to Google Engine and extract elevation

In [2]:
import ee
print(ee.__version__)

0.1.223


In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1wFRS2PCwqkpNRLktbJ2TGZGEhdtra2l6G04AwL37D-3yH4zA5HgbvQ

Successfully saved authorization token.


In [4]:
# Check/Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.924046, 27.987818])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8678


In [6]:
# Get the global Slope
slope = ee.Terrain.slope(dem); # here we have the information fro slope
from IPython.display import Image

# Display a thumbnail of global slope.
Image(url = slope.updateMask(slope.gt(0))
  .getThumbUrl({'min': 0, 'max': 5, 'dimensions': 712,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:
%%time


#get a copy of the geodatafame and add an empty column for the slope
gdf_c = gdf.copy()
gdf_c['slope'] = ""

#for some coordinates there is no slope info: bypass these rows and continue to the next set of coordinates
#add each slope to the corresponding place in the geodataframe
for x,y,i in zip(gdf_c.Longitude, gdf_c.Latitude, range(gdf_c.shape[0])):
    point = ee.Geometry.Point([x, y])
    try:
        gdf_c['slope'].iloc[i] = slope.sample(point, 30).first().get('slope').getInfo()
    except:
        continue

gdf_nan = gdf_c.replace('', np.nan)

In [ ]:
#check for missing slope data (from originally missing values)= NaN

import seaborn as sns
sns_plot = sns.heatmap(gdf_nan.isna(),yticklabels=False,cbar=False, cmap='viridis')

fig = sns_plot.get_figure()

fig.savefig('missing_values.png')

print("missed data for slope: {} datapoints, {}% of the data".format(gdf[gdf.slope.isna()].shape[0], round(gdf[gdf.slope.isna()].shape[0] / gdf.shape[0] *100,1)))

In [ ]:
gdf_clean = gdf_nan.dropna()

gdf_clean.to_csv('slope_gdf.csv')

In [ ]:
gdf_clean

In [ ]:
#probably we do not need that
#check the negative elevation values
gdf_neg = gdf_clean[gdf_clean.slope < 0]

from matplotlib.colors import LogNorm
plt.close('all')

f = plt.figure(figsize=(15,10)) 
ax = f.add_subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([-180, 180,-90, 90])
ax.coastlines()

img = ax.scatter(gdf_neg.Longitude, gdf_neg.Latitude, s=gdf_neg.VHRFC_LIS_FRD*10, c=gdf_neg.VHRFC_LIS_FRD, cmap=plt.cm.rainbow, norm=LogNorm())
ax.set_title('Places with negative elevation values and their flash rates', fontsize=20)
cb = plt.colorbar(img, orientation='horizontal', pad=0.05, aspect = 50) 
cb.set_label('Flash rate density (flashes / km2 year)', fontsize=14) 

plt.savefig('Map with negative elevation.png', bbox_inches='tight')

plt.show()

In [ ]:
gdf_neg.sort_values(by='slope')